# Homework 2
Провалидидировать и засабмитить следующие модели в виде классов с fit/recommend:
- Любую модель из библиотеки LightFM
- Какую-нибудь модель из библиотеки implicit из:
  - ALS
  - Item-to-Item kNN

# Imports

In [1]:
# turn off bad autocompleter
%config Completer.use_jedi = False
# magic for modules reloading 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from copy import deepcopy
from tqdm.auto import tqdm
# from scipy.stats import mode
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import gc
# import warnings
# warnings.filterwarnings("ignore")

from rectools import Columns
from rectools.dataset import Dataset, Interactions

# Load data
Обработка данных такая же как и в ДЗ 1. Поэтому часть клеток я опущу для краткости

In [3]:
interactions = pd.read_csv('data_full/interactions.csv')
users = pd.read_csv('data_full/users.csv')
items = pd.read_csv('data_full/items.csv')

In [5]:
interactions = interactions.rename(columns = {
    'last_watch_dt': Columns.Datetime,
    'total_dur': Columns.Weight,
})

In [6]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime])

# Validation

In [22]:
from rectools.metrics import MAP, Recall, calc_metrics
from validation import CrossValidator

/Users/ml003/miniconda3/envs/recom/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [23]:
from implicit.als import AlternatingLeastSquares

In [24]:
from models import ALS, LightFMCustom

In [64]:
K_RECOS = 10
random_state = 73

models = {
    'lf_basic'           : LightFMCustom(random_state=random_state),
    'lf_lr=.01_ncomp=20' : LightFMCustom(learning_rate=0.01, no_components=20, random_state=random_state),
    'lf_lr=.01_ncomp=40' : LightFMCustom(learning_rate=0.01, no_components=40, random_state=random_state),
    'lf_lr=.01_ncomp=100': LightFMCustom(learning_rate=0.01, no_components=100, random_state=random_state),
    'lf_loss=bpr'        : LightFMCustom(loss='bpr', random_state=random_state),
    'lf_max_sampled=20'  : LightFMCustom(max_sampled=20, random_state=random_state),
    
    'als_basic'              : ALS(random_state=random_state),
    'als_factors=10'         : ALS(factors=10, random_state=random_state),
    'als_factors=20'         : ALS(factors=20, random_state=random_state),
    'als_factors=40'         : ALS(factors=40, random_state=random_state),
    'als_regularization=1e-5': ALS(regularization=1e-5, random_state=random_state),
    'als_regularization=1e-3': ALS(regularization=1e-3, random_state=random_state),
    'als_regularization=1'   : ALS(regularization=1, random_state=random_state),
    'als_iterations=10'      : ALS(iterations=10, random_state=random_state),
    'als_iterations=30'      : ALS(iterations=30, random_state=random_state),
}

metrics = {
    "Recall@10": Recall(k=K_RECOS),
    "MAP@10": MAP(k=K_RECOS),
}

In [55]:
interactions.sort_values('datetime').datetime.iloc[[0, -1]]

1863186   2021-03-13
714056    2021-08-22
Name: datetime, dtype: datetime64[ns]

In [53]:
last_date = '2021-08-15'

In [54]:
cv = CrossValidator(
        last_date,
        n_folds = 3,
        n_days_per_fold = 2, # так как есть явный недельный цикл
        filter_already_seen = True,
        filter_cold_items = True,
        filter_cold_users = True,
    )

start_date: 2021-08-10 00:00:00
last_date: 2021-08-15 00:00:00
periods: 4
freq: 2D



In [56]:
interactions_test = interactions[
    (interactions.datetime >= '2021-08-09')  
]

In [66]:
%%time
df_results = cv.validate(
    models=models,
    metrics=metrics,
    interactions=Interactions(interactions_test), 
    k_recos=K_RECOS,
)

/Users/ml003/miniconda3/envs/recom/lib/python3.8/site-packages/rectools/dataset/interactions.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columns.Weight] = df[Columns.Weight].astype(float)
/Users/ml003/miniconda3/envs/recom/lib/python3.8/site-packages/rectools/dataset/interactions.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columns.Datetime] = df[Columns.Datetime].astype("datetime64[ns]")


Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/15 [00:00<?, ?it/s]

Fold: 0, Model: lf_basic


Recoms for users:   0%|          | 0/11970 [00:00<?, ?it/s]

...

  0%|          | 0/120493 [00:00<?, ?it/s]

Fold: 2, Model: als_iterations=30


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/120493 [00:00<?, ?it/s]

CPU times: user 18min 46s, sys: 2min 1s, total: 20min 47s
Wall time: 15min 18s


In [93]:
# Aggregate metrics by folds and compare models
pivot_results = df_results.drop(columns='fold').groupby(['model'], sort=False).agg(['mean', 'std'])
pivot_results.round(5).sort_values(by=[('MAP@10', 'mean'), ('Recall@10', 'mean')], ascending=False)

Recall@10            MAP@10         
                             mean      std     mean      std
model                                                       
als_regularization=1      0.08370  0.01434  0.03229  0.00414
als_iterations=30         0.07861  0.00346  0.02926  0.00201
als_basic                 0.06071  0.00435  0.02206  0.00153
als_factors=40            0.06068  0.00490  0.02197  0.00191
als_factors=20            0.06043  0.00028  0.02117  0.00118
als_regularization=1e-5   0.05255  0.00499  0.02036  0.00068
als_regularization=1e-3   0.05278  0.00497  0.02028  0.00076
als_factors=10            0.05916  0.00578  0.01895  0.00085
als_iterations=10         0.05118  0.00363  0.01850  0.00023
lf_basic                  0.00673  0.00915  0.00180  0.00240
lf_max_sampled=20         0.00673  0.00915  0.00180  0.00240
lf_lr=.01_ncomp=40        0.00499  0.00325  0.00133  0.00093
lf_lr=.01_ncomp=20        0.00368  0.00149  0.00092  0.00032
lf_lr=.01_ncomp=100       0.00216  0.00153  0.00062  0.00048
lf_loss=bpr               0.00033  0.00022  0.00009  0.00005

## One more try

In [73]:
K_RECOS = 10
random_state = 73

models = {
    'lf_loss=warp': LightFMCustom(loss='warp', random_state=random_state),
    'als_best': ALS(regularization=1, iterations=30, random_state=random_state),
}

metrics = {
    "Recall@10": Recall(k=K_RECOS),
    "MAP@10": MAP(k=K_RECOS),
}

In [74]:
cv = CrossValidator(
        last_date,
        n_folds = 3,
        n_days_per_fold = 2, # так как есть явный недельный цикл
        filter_already_seen = True,
        filter_cold_items = True,
        filter_cold_users = True,
    )

start_date: 2021-08-10 00:00:00
last_date: 2021-08-15 00:00:00
periods: 4
freq: 2D



In [75]:
%%time
df_results_2 = cv.validate(
    models=models,
    metrics=metrics,
    interactions=Interactions(interactions_test), 
    k_recos=K_RECOS,
#     item_features=item_features,
#     user_features=user_features,
#     cat_user_features=cat_user_features,
#     cat_item_features=cat_item_features,   
)

/Users/ml003/miniconda3/envs/recom/lib/python3.8/site-packages/rectools/dataset/interactions.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columns.Weight] = df[Columns.Weight].astype(float)
/Users/ml003/miniconda3/envs/recom/lib/python3.8/site-packages/rectools/dataset/interactions.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columns.Datetime] = df[Columns.Datetime].astype("datetime64[ns]")


Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/2 [00:00<?, ?it/s]

Fold: 0, Model: lf_loss=warp


Recoms for users:   0%|          | 0/11970 [00:00<?, ?it/s]

...

Recoms for users:   0%|          | 0/27651 [00:00<?, ?it/s]

Fold: 2, Model: als_best


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/120493 [00:00<?, ?it/s]

CPU times: user 3min 35s, sys: 22.6 s, total: 3min 58s
Wall time: 2min 41s


In [76]:
df_results_2

,fold,model,Recall@10,MAP@10
0,0,lf_loss=warp,0.134091,0.033415
1,0,als_best,0.057742,0.025939
2,1,lf_loss=warp,0.140780,0.034502
3,1,als_best,0.079990,0.031447
4,2,lf_loss=warp,0.147015,0.038028
5,2,als_best,0.085114,0.031088


In [77]:
# Aggregate metrics by folds and compare models
pivot_results_2 = df_results_2.drop(columns='fold').groupby(['model'], sort=False).agg(['mean', 'std'])
pivot_results_2.round(5).sort_values(by=[('MAP@10', 'mean'), ('Recall@10', 'mean')], ascending=False)

Recall@10            MAP@10         
                  mean      std     mean      std
model                                            
lf_loss=warp   0.14063  0.00646  0.03532  0.00241
als_best       0.07428  0.01455  0.02949  0.00308

ALS улучшить не получилось, а lightFM c warp лоссом лучше всех предыдущих моделей

# Load sample submission

In [78]:
submission_sample = pd.read_csv('data_full/sample_submission.csv')

In [81]:
cold_users = submission_sample.user_id[~submission_sample.user_id.isin(interactions.user_id)]

In [82]:
warm_users = submission_sample.user_id[submission_sample.user_id.isin(interactions.user_id)]

In [83]:
(cold_users.shape[0] + warm_users.shape[0]) / submission_sample.shape[0]

1.0

# Train final models

In [84]:
all_models = {
    'lightfm': LightFMCustom(loss='warp', random_state=random_state),
    'als': ALS(regularization=1, random_state=random_state),
}

In [86]:
dataset = Dataset.construct(interactions)

In [87]:
recoms = {}
for model_name, model in tqdm(all_models.items()):
    print(model_name)
    model = model.fit(dataset)
    recoms[model_name] = model.recommend(warm_users, dataset, K_RECOS, filter_viewed=True)

  0%|          | 0/2 [00:00<?, ?it/s]

lightfm


Recoms for users:   0%|          | 0/128170 [00:00<?, ?it/s]

als


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/962179 [00:00<?, ?it/s]

# Make submission

In [88]:
def make_submission(recoms):
    most_popular = recoms.item_id.value_counts().index[:10].tolist() # для холодных юзеров
    submission = pd.Series(
        [most_popular] * submission_sample.shape[0], 
        index=submission_sample['user_id'],
        name='item_id'
    )
    sub = recoms[['user_id', 'item_id']].groupby('user_id').agg(lambda x: list(x))
    submission.loc[sub.index] = sub.item_id
    return submission.reset_index()

In [89]:
for model_name, rec in tqdm(recoms.items()):
    sub = make_submission(rec)
    sub.to_csv(f'submissions/{model_name}.csv', index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

In [216]:
! head -5 data_full/sample_submission.csv

user_id,item_id
3,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"
11,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"
29,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"
30,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"


In [90]:
! head -5 submissions/lightfm.csv

user_id,item_id
3,"[7571, 142, 4880, 2657, 3734, 4151, 13865, 15297, 9728, 10440]"
11,"[12192, 142, 4880, 2657, 3734, 13865, 4151, 9728, 10440, 15297]"
29,"[9728, 10440, 15297, 13865, 3734, 4151, 2657, 4880, 142, 9996]"
30,"[12192, 142, 4880, 3734, 2657, 9728, 13865, 4151, 10440, 15297]"


In [91]:
! head -5 submissions/als.csv

user_id,item_id
3,"[9817, 15221, 14095, 8486, 8727, 8636, 14431, 1776, 101, 884]"
11,"[142, 11778, 1844, 14431, 8618, 8636, 4495, 12396, 4151, 14901]"
29,"[12192, 6809, 4740, 4151, 142, 1844, 13865, 9996, 3734, 9728]"
30,"[1261, 7160, 125, 3296, 1290, 14899, 6646, 9427, 2043, 561]"
